## Задача 3-2. Задача TSP: нижняя оценка Гельда—Карпа.

В этой задаче Вам предлагается релизовать алгоритм Гельда—Карпа для нижней оценки стоимости решения в задаче Euclidean TSP.

Сделайте следующее:
* Скачайте файл [`tsp-instances.zip`](https://github.com/dainiak/discrete-optimization-course/raw/master/tsp-instances.zip) и разархивируйте из него файлы со входами задачи TSP. Это в точности те же входные данные, что и в задании 3-1.
* Реализуйте функцию `lower_bound_tsp`. При этом можно пользоваться каким-нибудь стандартным алгоритмом построения минимального остовного дерева из библиотеки [`networkx`](https://networkx.github.io/), входящей в состав дистрибутива Anaconda.
* Запустите функцию `run_all()`, чтобы протестировать свой код, и напишите полученные, как следствия, верхние оценки погрешностей решений, которые были получены Вашими алгоритмами NN и NI при решении задания 3-1. Запишите свои выводы в 1-2 предложениях в последней ячейке ipynb-файла.

In [2]:
from typing import List, Tuple
from math import sqrt
from itertools import combinations, islice


def read_tsp_instance(filename: str) -> List[Tuple[int,int]]:
    with open(filename, 'r') as file:
        coordinates = []
        for line in file:
            line = line.strip().lower()
            if line.startswith('dimension'):
                coordinates = [(0, 0)] * int(line.split()[-1])
            tokens = line.split()
            if len(tokens) == 3 and tokens[0].isdecimal():
                tokens = line.split()
                coordinates[int(tokens[0])-1] = tuple(map(float, tokens[1:]))
        return coordinates


def euclidean_distance(point1: Tuple[int,int], point2: Tuple[int,int]) -> float:
    return sqrt((point1[0]-point2[0]) ** 2 + (point1[1]-point2[1]) ** 2)

In [4]:
import networkx as nx

In [19]:
def lower_bound_tsp(vertex_coordinates: List[Tuple[int,int]]) -> float:
    # Replace this trivial lower bound with Held—Karp:
    G = nx.Graph()
    n = len(vertex_coordinates)
    for i in range(n):
        for j in range(i + 1, n):
            G.add_edge(i, j, weight=euclidean_distance(vertex_coordinates[i], vertex_coordinates[j]))
    mst = nx.minimum_spanning_tree(G)
    res = sum([edge[-1]['weight'] for edge in mst.edges(data=True)])
    return res

In [31]:
import time
from os.path import exists

def run_all():
    instance_filenames = ['pr107.tsp', 'pr152.tsp', 'd198.tsp', 'pr439.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp']
    for filename in instance_filenames:
        if not exists('ht3-2/' + filename):
            print('File not found: “{}”. Skipping this instance.'.format(filename))
            continue
        instance = read_tsp_instance('ht3-2/' + filename)
        print('Instance {}…'.format(filename), end='')
        time_start = time.monotonic()
        bound = lower_bound_tsp(instance)
        time_nn = time.monotonic()-time_start
        print(' done in {:.2} seconds with lower bound {}'.format(time_nn, int(bound)))
        lower_bound[filename] = int(bound)

In [32]:
lower_bound = {}
run_all()

Instance pr107.tsp… done in 0.077 seconds with lower bound 34757
Instance pr152.tsp… done in 0.14 seconds with lower bound 59168
Instance d198.tsp… done in 0.24 seconds with lower bound 11767
Instance pr439.tsp… done in 1.3 seconds with lower bound 92193
Instance d493.tsp… done in 1.7 seconds with lower bound 29284
Instance d657.tsp… done in 3.5 seconds with lower bound 42490
Instance d2103.tsp… done in 3.7e+01 seconds with lower bound 76300


In [33]:
print(lower_bound)

{'pr107.tsp': 34757, 'pr152.tsp': 59168, 'd198.tsp': 11767, 'pr439.tsp': 92193, 'd493.tsp': 29284, 'd657.tsp': 42490, 'd2103.tsp': 76300}


Для сравнения: решения этих же графов с помощью Nearest Neighbour и Nearest Insertion:

Solving instance d198.tsp… done in 0.029 seconds with tour length 18620 using NN and in 0.16 seconds with tour length 17631 using NI

Solving instance d493.tsp… done in 0.18 seconds with tour length 43646 using NN and in 1.5 seconds with tour length 39982 using NI

Solving instance d657.tsp… done in 0.32 seconds with tour length 62176 using NN and in 3.9 seconds with tour length 57906 using NI

Solving instance d2103.tsp… done in 3.2 seconds with tour length 87468 using NN and in 1.4e+02 seconds with tour length 87530 using NI

Solving instance pr107.tsp… done in 0.0085 seconds with tour length 46678 using NN and in 0.05 seconds with tour length 51667 using NI

Solving instance pr152.tsp… done in 0.017 seconds with tour length 85702 using NN and in 0.093 seconds with tour length 88530 using NI

Solving instance pr439.tsp… done in 0.14 seconds with tour length 131282 using NN and in 1.1 seconds with tour length 130067 using NI


Запишем это всё в виде таблицы:

In [34]:
import pandas as pd
table = pd.DataFrame(columns = ['lower bound', 'NN', 'NI'])
for column in table.columns:
    table[column] = [None] * 7
table.index = ['pr107.tsp', 'pr152.tsp', 'd198.tsp', 'pr439.tsp', 'd493.tsp', 'd657.tsp', 'd2103.tsp']
table['lower bound'] = [lower_bound[x] for x in table.index]
NN = {'d198.tsp':18620, 'd493.tsp':43646, 'd657.tsp':62176, 'd2103.tsp':87648, 'pr107.tsp':46678, 'pr152.tsp':85702, 'pr439.tsp':131282}
NI = {'d198.tsp':17631, 'd493.tsp':39982, 'd657.tsp':57906, 'd2103.tsp':87530, 'pr107.tsp':51667, 'pr152.tsp':88530, 'pr439.tsp':130067}
table['NN'] = [NN[x] for x in table.index]
table['NI'] = [NI[x] for x in table.index]
table.head(7)

,lower bound,NN,NI
pr107.tsp,34757,46678,51667
pr152.tsp,59168,85702,88530
d198.tsp,11767,18620,17631
pr439.tsp,92193,131282,130067
d493.tsp,29284,43646,39982
d657.tsp,42490,62176,57906
d2103.tsp,76300,87648,87530


## Выводы
Алгоритмы NN и NI дают результаты не более чем в 1,5 раза хуже нижней оценки Гельда-Карпа. Значит, они решают задачу довольно эффективно